In [ ]:
!pip install backtrader

     |████████████████████████████████| 410 kB 26.0 MB/s 


In [ ]:
import datetime
import os.path  
import sys  
import backtrader as bt
import pandas as pd
from datetime import datetime
import backtrader.feeds as btfeed
import backtrader.plot
import matplotlib as plt
%matplotlib inline

In [ ]:
# Create a Stratey
class BOLLStrat(bt.Strategy):
    
    params = (
        ("period", 10),
        ("devfactor", 1),
        ("size", 0.1),
        ('movav', bt.ind.MovAv.Simple),
        ("debug", False)
        )

    def log(self, txt, dt=None):
        dt = self.datas[0].datetime.datetime(0)
        #dt = self.datas[0].datetime.date(0), self.datas[0].datetime.time(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.redline = None
        self.blueline = None
        self.bar_executed = 0
        self.duration = 0
        self.cum_pnl = 0
        self.df = pd.read_csv("/content/Targets (1).csv")
        self.df['timestamp'] = pd.to_datetime(self.df.timestamp)
        #self.price_bb = bt.indicators.BBands(self.datas[0].close, period=self.p.period, devfactor=self.p.devfactor, movav = self.p.movav)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return


        if order.status == order.Completed:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     ))

                self.buyprice = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          ))
                #self.sellprice = order.executed.price
            self.bar_executed = len(self)

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        self.cum_pnl += trade.pnl
        #self.log(f"The cumulative PNL is {self.cum_pnl}")

    

    def next(self):
        
        # if self.data.close <= self.price_bb.bot and self.position.size == 0:
        #   self.order = self.buy(exectype=bt.Order.Market, size = self.p.size)

        # if self.data.close >= self.price_bb.top and self.position.size > 0:
        #   self.order = self.sell(exectype=bt.Order.Market, size = self.p.size)
        
        #print("backtrader time -- ", self.datas[0].datetime.datetime())
        #print("DF time -- ", self.df['timestamp'][0])


        # if (str(self.datas[0].datetime.datetime())) in (pd.to_datetime(self.df['timestamp'].values)):
        #   #print("AAAAYOOOO")  
        #   if self.df.loc[self.df['timestamp'] == str(self.datas[0].datetime.datetime())]['Forecast_ARIMAX'].iloc[0] > 0.003 and self.position.size == 0:
        #     self.order = self.buy(exectype=bt.Order.Market, size = self.p.size)

        #   self.duration = len(self) - self.bar_executed + 1
        #   if self.duration >= 1 and self.position.size > 0:
        #     self.order = self.sell(exectype=bt.Order.Market, size = self.p.size)


        if (str(self.datas[0].datetime.datetime())) in (pd.to_datetime(self.df['timestamp'].values)):
          #print("AAAAYOOOO")  
          if self.df.loc[self.df['timestamp'] == str(self.datas[0].datetime.datetime())]['Forecast_ARIMAX'].iloc[0] < 0 and self.position.size == 0:
            self.order = self.sell(exectype=bt.Order.Market, size = self.p.size)

          self.duration = len(self) - self.bar_executed + 1
          if self.duration >= 1 and self.position.size < 0:
            self.order = self.buy(exectype=bt.Order.Market, size = self.p.size)

          #elif self.df.loc[self.df['timestamp'] == str(self.current_datetime)]['preds'].iloc[0] < 0 and self.position.size == 0:   


In [ ]:
cerebro = bt.Cerebro()

cerebro.broker.set_cash(1000000.0)
 
data = bt.feeds.GenericCSVData(
    dataname='/content/BTC_OHLCV.csv',
    fromdate=datetime(2022, 3, 1, 1, 30),
    todate=datetime(2022, 5, 4, 23, 30),
    nullvalue=0.0,
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=1,
    datetime=0,
    open = 1,
    high = 2,
    low = 3,
    close = 4,
    volume = 5, 
    openinterest=-1,
    reverse=False)
 
cerebro.adddata(data)

cerebro.addstrategy(BOLLStrat)


print("Starting Portfolio", cerebro.broker.get_value())

cerebro.run()

print("Final Portfolio", cerebro.broker.getvalue())

Starting Portfolio 1000000.0
2022-03-01T05:00:00, SELL EXECUTED, Price: 43369.98, Cost: -4337.00
2022-03-01T06:00:00, BUY EXECUTED, Price: 43231.75, Cost: -4337.00
2022-03-01T06:00:00, OPERATION PROFIT, GROSS 13.82, NET 13.82
2022-03-01T07:00:00, SELL EXECUTED, Price: 43464.00, Cost: -4346.40
2022-03-01T08:00:00, BUY EXECUTED, Price: 43522.27, Cost: -4346.40
2022-03-01T08:00:00, OPERATION PROFIT, GROSS -5.83, NET -5.83
2022-03-01T09:00:00, SELL EXECUTED, Price: 43553.85, Cost: -4355.39
2022-03-01T10:00:00, BUY EXECUTED, Price: 43255.12, Cost: -4355.39
2022-03-01T10:00:00, OPERATION PROFIT, GROSS 29.87, NET 29.87
2022-03-01T11:00:00, SELL EXECUTED, Price: 43576.45, Cost: -4357.64
2022-03-01T12:00:00, BUY EXECUTED, Price: 43623.33, Cost: -4357.64
2022-03-01T12:00:00, OPERATION PROFIT, GROSS -4.69, NET -4.69
2022-03-01T13:00:00, SELL EXECUTED, Price: 44690.55, Cost: -4469.06
2022-03-01T14:00:00, BUY EXECUTED, Price: 44470.67, Cost: -4469.06
2022-03-01T14:00:00, OPERATION PROFIT, GROSS 21.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Gemini_BTCUSD_1h.csv")
df = df.stack().reset_index()
df.columns = df.iloc[0]
df = df[1:]
df = df[["date", "open", "high", "low", "Volume BTC", "close"]]
df = df.rename({"Volume BTC" : "volume", "date" : "timestamp"}, axis = 1)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values("timestamp")
df = df.reset_index(drop = True)
#df = df[df["timestamp"] >= pd.to_datetime("2021-01-22")]
df = df.reset_index(drop = True)
df = df.dropna()
df.to_csv("/content/BTC_OHLCV.csv", index = False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/btc_latest_data.csv")
df

,timestamp,open,high,low,volume,close
0,2015-10-08 13:00:00,0.000000,245.000000,0.000000,0.606654,245.000000
1,2015-10-08 14:00:00,245.000000,245.000000,244.500000,4.453649,245.000000
2,2015-10-08 15:00:00,245.000000,245.000000,244.920000,3.016926,244.920000
3,2015-10-08 16:00:00,244.920000,244.920000,244.250000,3.895252,244.250000
4,2015-10-08 17:00:00,244.250000,244.990000,244.020000,3.920632,244.990000
...,...,...,...,...,...,...
57820,2022-05-13 19:00:00,30021.777667,30117.873566,29987.820833,27.643384,30028.296537
57821,2022-05-13 20:00:00,29929.955480,30005.285968,29894.149500,32.276690,29922.025833
57822,2022-05-13 21:00:00,29876.871587,29956.233238,29857.959000,19.595915,29883.118870
57823,2022-05-13 22:00:00,29909.401167,29980.443894,29884.236167,18.458942,29910.547259


In [ ]:
df2 = pd.read_csv("/content/btc_latest_data.csv")
df2 = df2.rename({"o" : "open", "h" : "high", "l" : "low", "c" : "close", "v" : "volume"}, axis = 1)
df2 = df2[["timestamp", "open", "high", "low", "volume", "close"]]
df2["timestamp"] = pd.to_datetime(df2["timestamp"])
df_all = df.append(df2)
df_all

,timestamp,open,high,low,volume,close
0,2015-10-08 13:00:00,0.0,245.0,0.0,0.60665438,245.0
1,2015-10-08 14:00:00,245.0,245.0,244.5,4.453648931,245.0
2,2015-10-08 15:00:00,245.0,245.0,244.92,3.016925828,244.92
3,2015-10-08 16:00:00,244.92,244.92,244.25,3.89525246,244.25
4,2015-10-08 17:00:00,244.25,244.99,244.02,3.920632003,244.99
...,...,...,...,...,...,...
163,2022-05-13 19:00:00,30021.777667,30117.873566,29987.820833,27.643384,30028.296537
164,2022-05-13 20:00:00,29929.95548,30005.285968,29894.1495,32.27669,29922.025833
165,2022-05-13 21:00:00,29876.871587,29956.233238,29857.959,19.595915,29883.11887
166,2022-05-13 22:00:00,29909.401167,29980.443894,29884.236167,18.458942,29910.547259


In [ ]:
df_all.to_csv("/content/btc_latest_data.csv", index = False)
!cp /content/btc_latest_data.csv /content/drive/MyDrive

In [ ]:
df_targets = pd.read_csv("/content/Targets (1).csv")
df_targets

,timestamp,close,Forecast_ARIMAX,Actual_target,Pred_target
0,2021-09-16 09:00:00,-0.002114,0.007218,0,1
1,2021-09-16 10:00:00,0.002085,0.007062,1,1
2,2021-09-16 11:00:00,0.002841,0.002585,1,1
3,2021-09-16 12:00:00,-0.012158,-0.002134,0,0
4,2021-09-16 13:00:00,0.002645,0.007244,1,1
...,...,...,...,...,...
5519,2022-05-04 19:00:00,-0.000851,-0.013983,0,0
5520,2022-05-04 20:00:00,-0.001591,-0.009422,0,0
5521,2022-05-04 21:00:00,-0.001570,-0.003333,0,0
5522,2022-05-04 22:00:00,0.000014,0.000431,1,1
